In [1]:
import os
import pickle
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import pearsonr
print(pd.__version__)

1.3.5


In [2]:
def age_debiasing(predicted_ages, chronological_ages):
    
    predicted_ages = predicted_ages.to_numpy()
    chronological_ages = chronological_ages.to_numpy()
    
    ones = np.ones(len(chronological_ages))
    chronological_ages_squared = np.square(chronological_ages)
    chronological_ages_confounds = np.array([ones, chronological_ages])
    
    ones_predicted = np.ones(len(predicted_ages))
    predicted_ages_squared = np.square(predicted_ages)
    predicted_ages_confounds = np.array([ones_predicted, predicted_ages])
    
    predicted_ages = np.reshape(predicted_ages, (len(predicted_ages), 1) )
    beta = np.dot( np.linalg.pinv(chronological_ages_confounds.T), predicted_ages )

    corrected_predicted_ages = (predicted_ages.flatten() - beta[0])/beta[1]
    
    return corrected_predicted_ages

In [3]:
T1_nonlinear="1"
T1_linear="2"
jacobian="56"
vbm="4"

T2_nonlinear="5"
T2_lesions="57"
swi="6"

rsfmri_0="7"
rsfmri_1="8"
rsfmri_2="9"
rsfmri_3="10"
rsfmri_4="11"
rsfmri_5="12"
rsfmri_6="13"
rsfmri_7="14"
rsfmri_8="15"
rsfmri_9="16"
rsfmri_10="17"
rsfmri_11="18"
rsfmri_12="19"
rsfmri_13="20"
rsfmri_14="21"
rsfmri_15="22"
rsfmri_16="23"
rsfmri_17="24"
rsfmri_18="25"
rsfmri_19="26"
rsfmri_20="27"
rsfmri_21="28"
rsfmri_22="29"
rsfmri_23="30"
rsfmri_24="31"

tfmri_1="32"
tfmri_2="33"
tfmri_5="34"

tfmri_c_1="35"
tfmri_c_2="36"
tfmri_c_5="37"

tracts="3"

tbss_FA_s="38"
tbss_ICVF_s="39"
tbss_ISOVF_s="40"
tbss_L1_s="41"
tbss_L2_s="42"
tbss_L3_s="43"
tbss_MD_s="44"
tbss_MO_s="45"
tbss_OD_s="46"

tbss_FA="47"
tbss_ICVF="48"
tbss_ISOVF="49"
tbss_L1="50"
tbss_L2="51"
tbss_L3="52"
tbss_MD="53"
tbss_MO="54"
tbss_OD="55"

dfs = [

T1_nonlinear,
T1_linear,
jacobian,
vbm,

T2_nonlinear,
T2_lesions,
swi,

rsfmri_0,
rsfmri_1,
rsfmri_2,
rsfmri_3,
rsfmri_4,
rsfmri_5,
rsfmri_6,
rsfmri_7,
rsfmri_8,
rsfmri_9,
rsfmri_10,
rsfmri_11,
rsfmri_12,
rsfmri_13,
rsfmri_14,
rsfmri_15,
rsfmri_16,
rsfmri_17,
rsfmri_18,
rsfmri_19,
rsfmri_20,
rsfmri_21,
rsfmri_22,
rsfmri_23,
rsfmri_24,

tfmri_1,
tfmri_2,
tfmri_5,

tfmri_c_1,
tfmri_c_2,
tfmri_c_5,

tracts,

tbss_FA_s,
tbss_ICVF_s,
tbss_ISOVF_s,
tbss_L1_s,
tbss_L2_s,
tbss_L3_s,
tbss_MD_s,
tbss_MO_s,
tbss_OD_s,

tbss_FA,
tbss_ICVF,
tbss_ISOVF,
tbss_L1,
tbss_L2,
tbss_L3,
tbss_MD,
tbss_MO,
tbss_OD
]

names = [

'T1_nonlinear',
'T1_linear',
'jacobian',
'vbm',

'T2_nonlinear',
'T2_lesions',
'swi',

'rsfmri_0',
'rsfmri_1',
'rsfmri_2',
'rsfmri_3',
'rsfmri_4',
'rsfmri_5',
'rsfmri_6',
'rsfmri_7',
'rsfmri_8',
'rsfmri_9',
'rsfmri_10',
'rsfmri_11',
'rsfmri_12',
'rsfmri_13',
'rsfmri_14',
'rsfmri_15',
'rsfmri_16',
'rsfmri_17',
'rsfmri_18',
'rsfmri_19',
'rsfmri_20',
'rsfmri_21',
'rsfmri_22',
'rsfmri_23',
'rsfmri_24',

'tfmri_1',
'tfmri_2',
'tfmri_5',

'tfmri_c_1',
'tfmri_c_2',
'tfmri_c_5',

'tracts',

'tbss_FA_s',
'tbss_ICVF_s',
'tbss_ISOVF_s',
'tbss_L1_s',
'tbss_L2_s',
'tbss_L3_s',
'tbss_MD_s',
'tbss_MO_s',
'tbss_OD_s',

'tbss_FA',
'tbss_ICVF',
'tbss_ISOVF',
'tbss_L1',
'tbss_L2',
'tbss_L3',
'tbss_MD',
'tbss_MO',
'tbss_OD'
]

dfs1, dfs2, dfs3 = [], [], []

dfs1 = dfs.copy()
dfs2 = dfs.copy()
dfs3 = dfs.copy()

for idx in range(len(dfs)):
    
    dfs1[idx] = '../predictions/AM5-' + dfs1[idx] + '_predictions/output_statistics_validation.csv'
    dfs2[idx] = '../predictions/AM6-' + dfs2[idx] + '_predictions/output_statistics_validation.csv'
    dfs3[idx] = '../predictions/AM7-' + dfs3[idx] + '_predictions/output_statistics_validation.csv'
    
for idx in range(len(dfs)):
    dfs1[idx] = pd.read_csv(dfs1[idx])
    dfs2[idx] = pd.read_csv(dfs2[idx])
    dfs3[idx] = pd.read_csv(dfs3[idx])

In [4]:
for idx in range(len(dfs)):
    dfs[idx] = pd.merge(dfs1[idx], dfs2[idx], on='Unnamed: 0', how='inner')
    dfs[idx] = pd.merge(dfs[idx], dfs3[idx], on='Unnamed: 0', how='inner')
    
    dfs[idx].rename({
        'target_age_x': 'target_age_original',
        'output_age_x': 'output_age_1',
        'age_delta_x': 'age_delta_1',
        'loss_x': 'loss_1',
        'target_age_y': 'target_age_2',
        'output_age_y': 'output_age_2',
        'age_delta_y': 'age_delta_2',
        'loss_y': 'loss_2',
        'target_age': 'target_age_3',
        'output_age': 'output_age_3',
        'age_delta': 'age_delta_3',
        'loss': 'loss_3',
    }, axis=1, inplace=True)
    
    dfs[idx].drop(['target_age_2', 'target_age_3'], axis=1, inplace=True)
    
    dfs[idx].rename({
        'target_age_original': 'target_age',
    }, axis=1, inplace=True)
    
    dfs[idx]['mean_output_age'] = dfs[idx][['output_age_1', 'output_age_2', 'output_age_3']].mean(axis=1)
    dfs[idx]['mean_age_delta'] = dfs[idx]['mean_output_age'] - dfs[idx]['target_age']

In [5]:
for idx in range(len(dfs)):    
    dfs[idx]['output_age_decon'] = age_debiasing(dfs[idx].mean_output_age, dfs[idx].target_age) 
    dfs[idx]['age_delta_decon'] = dfs[idx].output_age_decon - dfs[idx].target_age

In [6]:
print("{:13} : {:15}  |  {:13}  |  {:13}  |  {:13}".format('Modality', 'Run1', "Run2", "Run3", "MEAN"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} : {:7.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  MEAN --- {:.3f} ± {:.3f}".format(
        names[idx],
        np.abs(dfs[idx].age_delta_1).mean(), np.abs(dfs[idx].age_delta_1).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_2).mean(), np.abs(dfs[idx].age_delta_2).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_3).mean(), np.abs(dfs[idx].age_delta_3).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
    )
         )

Modality      : Run1             |  Run2           |  Run3           |  MEAN         
-----------------------------------------------------------------------------------------------
T1_nonlinear  :   2.702 ± 0.062  |  2.775 ± 0.063  |  2.742 ± 0.060  |  MEAN --- 2.504 ± 0.056
T1_linear     :   2.856 ± 0.065  |  2.832 ± 0.065  |  2.701 ± 0.062  |  MEAN --- 2.564 ± 0.059
jacobian      :   3.866 ± 0.085  |  3.765 ± 0.084  |  3.677 ± 0.083  |  MEAN --- 3.604 ± 0.079
vbm           :   3.053 ± 0.070  |  3.135 ± 0.070  |  3.137 ± 0.071  |  MEAN --- 2.907 ± 0.064
T2_nonlinear  :   2.676 ± 0.061  |  2.617 ± 0.059  |  2.600 ± 0.059  |  MEAN --- 2.434 ± 0.055
T2_lesions    :   4.475 ± 0.096  |  4.367 ± 0.095  |  4.502 ± 0.097  |  MEAN --- 4.380 ± 0.094
swi           :   3.612 ± 0.082  |  3.568 ± 0.084  |  3.549 ± 0.085  |  MEAN --- 3.300 ± 0.076
rsfmri_0      :   5.053 ± 0.114  |  5.376 ± 0.116  |  4.878 ± 0.108  |  MEAN --- 4.796 ± 0.106
rsfmri_1      :   5.096 ± 0.110  |  5.048 ± 0.110  |  4.85

In [7]:
print("{:13} : {:14}  |  {:13}  ".format('Modality', 'MEAN (MAE)', "MEAN DECONF"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
    )
         )

Modality      : MEAN (MAE)      |  MEAN DECONF    
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  2.504 ± 0.056  |   2.743 ± 0.061 
T1_linear     :  2.564 ± 0.059  |   2.822 ± 0.065 
jacobian      :  3.604 ± 0.079  |   4.414 ± 0.098 
vbm           :  2.907 ± 0.064  |   3.269 ± 0.074 
T2_nonlinear  :  2.434 ± 0.055  |   2.639 ± 0.061 
T2_lesions    :  4.380 ± 0.094  |   6.088 ± 0.136 
swi           :  3.300 ± 0.076  |   3.927 ± 0.092 
rsfmri_0      :  4.796 ± 0.106  |   7.558 ± 0.177 
rsfmri_1      :  4.756 ± 0.103  |   7.307 ± 0.170 
rsfmri_2      :  5.148 ± 0.109  |   8.821 ± 0.208 
rsfmri_3      :  5.134 ± 0.111  |   9.083 ± 0.208 
rsfmri_4      :  4.900 ± 0.109  |   8.010 ± 0.181 
rsfmri_5      :  4.635 ± 0.103  |   6.961 ± 0.165 
rsfmri_6      :  5.308 ± 0.110  |   9.656 ± 0.224 
rsfmri_7      :  5.525 ± 0.115  |   11.524 ± 0.262 
rsfmri_8      :  4.962 ± 0.104  |   7.828 ± 0.183 
rsfmri_9      :  4.916 ± 0.109  |   

In [8]:
mae_weighting = 37.137671232876755 # for male subjects
# mae_weighting = 36.28150684931529 # for female subjects


print("{:13} : {:14}  |  {:15}  |  {:7}  |  {:13} ".format('Modality', 'MAE', "RMSE", 
                                                            "CORR (R)", "MAE weighted"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:5.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).std()/np.sqrt(len(dfs[idx]))),
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0],
        np.abs(dfs[idx].mean_age_delta).mean() / mae_weighting,
    )
         )

Modality      : MAE             |  RMSE             |  CORR (R)  |  MAE weighted  
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  2.504 ± 0.056  |   3.104 ± 0.626   |   0.912   |   0.067 
T1_linear     :  2.564 ± 0.059  |   3.215 ± 0.660   |   0.905   |   0.069 
jacobian      :  3.604 ± 0.079  |   4.448 ± 0.884   |   0.810   |   0.097 
vbm           :  2.907 ± 0.064  |   3.597 ± 0.727   |   0.880   |   0.078 
T2_nonlinear  :  2.434 ± 0.055  |   3.044 ± 0.618   |   0.916   |   0.066 
T2_lesions    :  4.380 ± 0.094  |   5.362 ± 1.049   |   0.707   |   0.118 
swi           :  3.300 ± 0.076  |   4.151 ± 0.872   |   0.836   |   0.089 
rsfmri_0      :  4.796 ± 0.106  |   5.933 ± 1.189   |   0.621   |   0.129 
rsfmri_1      :  4.756 ± 0.103  |   5.855 ± 1.158   |   0.634   |   0.128 
rsfmri_2      :  5.148 ± 0.109  |   6.276 ± 1.211   |   0.561   |   0.139 
rsfmri_3      :  5.134 ± 0.111  |   6.307 ± 1.250   |   0.554   |   0.1

In [9]:
maes=[]
prs = []
for idx in range(len(dfs)):
    mae = np.abs(dfs[idx].mean_age_delta).mean()
    maes.append(mae)
    pr = pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0]
    prs.append(pr)
    
print(pearsonr(maes, prs)[0])

-0.993209963162551


In [10]:
print("{:13} : {:14}  |  {:15}  |  {:13} ".format('Modality', 'MAE (DECON)', "RMSE (DECON)", "PR (DECON)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ".format(
        names[idx],
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx]))),
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
    )
         )

Modality      : MAE (DECON)     |  RMSE (DECON)     |  PR (DECON)    
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  2.743 ± 0.061  |   3.404 ± 0.697   |   0.912 
T1_linear     :  2.822 ± 0.065  |   3.550 ± 0.747   |   0.905 
jacobian      :  4.414 ± 0.098  |   5.477 ± 1.114   |   0.810 
vbm           :  3.269 ± 0.074  |   4.078 ± 0.825   |   0.880 
T2_nonlinear  :  2.639 ± 0.061  |   3.324 ± 0.690   |   0.916 
T2_lesions    :  6.088 ± 0.136  |   7.565 ± 1.535   |   0.707 
swi           :  3.927 ± 0.092  |   4.960 ± 1.050   |   0.836 
rsfmri_0      :  7.558 ± 0.177  |   9.550 ± 2.011   |   0.621 
rsfmri_1      :  7.307 ± 0.170  |   9.221 ± 1.973   |   0.634 
rsfmri_2      :  8.821 ± 0.208  |   11.177 ± 2.328   |   0.561 
rsfmri_3      :  9.083 ± 0.208  |   11.378 ± 2.374   |   0.554 
rsfmri_4      :  8.010 ± 0.181  |   9.992 ± 2.099   |   0.604 
rsfmri_5      :  6.961 ± 0.165  |   8.836 ± 1.859   |   0.650 
rsfmri_6     

# Saving Dataframe for Later Use

In [11]:
d = {'modality':names, 'dataframe':dfs}
df = pd.DataFrame(data=d)

In [12]:
df['dataframe'].iloc[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23612373,73.229110,72.245190,-0.983916,0.968091,75.021480,1.792367,3.212580,76.916565,3.687455,13.597327,74.727745,1.498635,76.883969,3.654860
1,23018021,64.545548,60.477158,-4.068390,16.551800,61.176723,-3.368824,11.348978,63.002680,-1.542866,2.380436,61.552187,-2.993361,60.955985,-3.589563
2,25398623,67.725457,66.032880,-1.692574,2.864807,65.932846,-1.792611,3.213453,64.098270,-3.627190,13.156507,65.354665,-2.370791,65.552816,-2.172640
3,21140289,58.197603,57.446762,-0.750841,0.563762,59.883380,1.685778,2.841848,55.333614,-2.863988,8.202430,57.554585,-0.643017,56.123269,-2.074334
4,25346433,62.627968,63.659073,1.031105,1.063177,67.508200,4.880234,23.816680,66.946780,4.318809,18.652114,66.038018,3.410050,66.378924,3.750956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,21422666,51.102397,49.896027,-1.206371,1.455330,49.362335,-1.740062,3.027816,53.252010,2.149613,4.620836,50.836791,-0.265607,48.002100,-3.100297
1087,21609582,73.242580,71.538100,-1.704479,2.905248,70.784570,-2.458011,6.041819,70.616974,-2.625606,6.893807,70.979881,-2.262699,72.353162,-0.889418
1088,22882299,53.880479,53.860794,-0.019685,0.000388,51.119675,-2.760805,7.622043,52.539883,-1.340597,1.797200,52.506784,-1.373695,50.020962,-3.859518
1089,22011925,68.198770,66.386430,-1.812342,3.284582,63.151203,-5.047567,25.477936,65.467740,-2.731028,7.458512,65.001791,-3.196979,65.126225,-3.072545


In [13]:
df

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [14]:
type(df.dataframe.iloc[0])

pandas.core.frame.DataFrame

In [15]:
df.iloc[0].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23612373,73.229110,72.245190,-0.983916,0.968091,75.021480,1.792367,3.212580,76.916565,3.687455,13.597327,74.727745,1.498635,76.883969,3.654860
1,23018021,64.545548,60.477158,-4.068390,16.551800,61.176723,-3.368824,11.348978,63.002680,-1.542866,2.380436,61.552187,-2.993361,60.955985,-3.589563
2,25398623,67.725457,66.032880,-1.692574,2.864807,65.932846,-1.792611,3.213453,64.098270,-3.627190,13.156507,65.354665,-2.370791,65.552816,-2.172640
3,21140289,58.197603,57.446762,-0.750841,0.563762,59.883380,1.685778,2.841848,55.333614,-2.863988,8.202430,57.554585,-0.643017,56.123269,-2.074334
4,25346433,62.627968,63.659073,1.031105,1.063177,67.508200,4.880234,23.816680,66.946780,4.318809,18.652114,66.038018,3.410050,66.378924,3.750956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,21422666,51.102397,49.896027,-1.206371,1.455330,49.362335,-1.740062,3.027816,53.252010,2.149613,4.620836,50.836791,-0.265607,48.002100,-3.100297
1087,21609582,73.242580,71.538100,-1.704479,2.905248,70.784570,-2.458011,6.041819,70.616974,-2.625606,6.893807,70.979881,-2.262699,72.353162,-0.889418
1088,22882299,53.880479,53.860794,-0.019685,0.000388,51.119675,-2.760805,7.622043,52.539883,-1.340597,1.797200,52.506784,-1.373695,50.020962,-3.859518
1089,22011925,68.198770,66.386430,-1.812342,3.284582,63.151203,-5.047567,25.477936,65.467740,-2.731028,7.458512,65.001791,-3.196979,65.126225,-3.072545


In [16]:
df.iloc[0].modality

'T1_nonlinear'

In [17]:
type(df.iloc[0].dataframe)

pandas.core.frame.DataFrame

In [18]:
df

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [19]:
df.to_pickle('male_validation.pkl', protocol = 4)

In [20]:
df2 = pd.read_pickle('male_validation.pkl')

In [21]:
df2

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [22]:
df2.iloc[0].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23612373,73.229110,72.245190,-0.983916,0.968091,75.021480,1.792367,3.212580,76.916565,3.687455,13.597327,74.727745,1.498635,76.883969,3.654860
1,23018021,64.545548,60.477158,-4.068390,16.551800,61.176723,-3.368824,11.348978,63.002680,-1.542866,2.380436,61.552187,-2.993361,60.955985,-3.589563
2,25398623,67.725457,66.032880,-1.692574,2.864807,65.932846,-1.792611,3.213453,64.098270,-3.627190,13.156507,65.354665,-2.370791,65.552816,-2.172640
3,21140289,58.197603,57.446762,-0.750841,0.563762,59.883380,1.685778,2.841848,55.333614,-2.863988,8.202430,57.554585,-0.643017,56.123269,-2.074334
4,25346433,62.627968,63.659073,1.031105,1.063177,67.508200,4.880234,23.816680,66.946780,4.318809,18.652114,66.038018,3.410050,66.378924,3.750956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,21422666,51.102397,49.896027,-1.206371,1.455330,49.362335,-1.740062,3.027816,53.252010,2.149613,4.620836,50.836791,-0.265607,48.002100,-3.100297
1087,21609582,73.242580,71.538100,-1.704479,2.905248,70.784570,-2.458011,6.041819,70.616974,-2.625606,6.893807,70.979881,-2.262699,72.353162,-0.889418
1088,22882299,53.880479,53.860794,-0.019685,0.000388,51.119675,-2.760805,7.622043,52.539883,-1.340597,1.797200,52.506784,-1.373695,50.020962,-3.859518
1089,22011925,68.198770,66.386430,-1.812342,3.284582,63.151203,-5.047567,25.477936,65.467740,-2.731028,7.458512,65.001791,-3.196979,65.126225,-3.072545


In [23]:
df2.iloc[0].modality

'T1_nonlinear'

In [24]:
type(df2.iloc[0].dataframe)

pandas.core.frame.DataFrame

# Test what happens if debiasing before deconfounding

In [25]:
dfs[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23612373,73.229110,72.245190,-0.983916,0.968091,75.021480,1.792367,3.212580,76.916565,3.687455,13.597327,74.727745,1.498635,76.883969,3.654860
1,23018021,64.545548,60.477158,-4.068390,16.551800,61.176723,-3.368824,11.348978,63.002680,-1.542866,2.380436,61.552187,-2.993361,60.955985,-3.589563
2,25398623,67.725457,66.032880,-1.692574,2.864807,65.932846,-1.792611,3.213453,64.098270,-3.627190,13.156507,65.354665,-2.370791,65.552816,-2.172640
3,21140289,58.197603,57.446762,-0.750841,0.563762,59.883380,1.685778,2.841848,55.333614,-2.863988,8.202430,57.554585,-0.643017,56.123269,-2.074334
4,25346433,62.627968,63.659073,1.031105,1.063177,67.508200,4.880234,23.816680,66.946780,4.318809,18.652114,66.038018,3.410050,66.378924,3.750956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,21422666,51.102397,49.896027,-1.206371,1.455330,49.362335,-1.740062,3.027816,53.252010,2.149613,4.620836,50.836791,-0.265607,48.002100,-3.100297
1087,21609582,73.242580,71.538100,-1.704479,2.905248,70.784570,-2.458011,6.041819,70.616974,-2.625606,6.893807,70.979881,-2.262699,72.353162,-0.889418
1088,22882299,53.880479,53.860794,-0.019685,0.000388,51.119675,-2.760805,7.622043,52.539883,-1.340597,1.797200,52.506784,-1.373695,50.020962,-3.859518
1089,22011925,68.198770,66.386430,-1.812342,3.284582,63.151203,-5.047567,25.477936,65.467740,-2.731028,7.458512,65.001791,-3.196979,65.126225,-3.072545


In [26]:
for idx in range(len(dfs)):
    dfs[idx]['output_age_decon_1'] = age_debiasing(dfs[idx].output_age_1, dfs[idx].target_age)
    dfs[idx]['output_age_decon_2'] = age_debiasing(dfs[idx].output_age_2, dfs[idx].target_age) 
    dfs[idx]['output_age_decon_3'] = age_debiasing(dfs[idx].output_age_3, dfs[idx].target_age) 
    dfs[idx]['mean_output_age_decon'] = dfs[idx][['output_age_decon_1', 'output_age_decon_2', 'output_age_decon_3']].mean(axis=1)
    dfs[idx]['mean_age_delta_decon'] = dfs[idx]['mean_output_age_decon'] - dfs[idx]['target_age']

In [27]:
dfs[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon,output_age_decon_1,output_age_decon_2,output_age_decon_3,mean_output_age_decon,mean_age_delta_decon
0,23612373,73.229110,72.245190,-0.983916,0.968091,75.021480,1.792367,3.212580,76.916565,3.687455,13.597327,74.727745,1.498635,76.883969,3.654860,74.596803,76.936978,79.092940,76.875574,3.646464
1,23018021,64.545548,60.477158,-4.068390,16.551800,61.176723,-3.368824,11.348978,63.002680,-1.542866,2.380436,61.552187,-2.993361,60.955985,-3.589563,60.206278,60.400082,62.260680,60.955680,-3.589868
2,25398623,67.725457,66.032880,-1.692574,2.864807,65.932846,-1.792611,3.213453,64.098270,-3.627190,13.156507,65.354665,-2.370791,65.552816,-2.172640,67.000087,66.081042,63.586065,65.555731,-2.169725
3,21140289,58.197603,57.446762,-0.750841,0.563762,59.883380,1.685778,2.841848,55.333614,-2.863988,8.202430,57.554585,-0.643017,56.123269,-2.074334,56.500562,58.855246,52.983061,56.112956,-2.084646
4,25346433,62.627968,63.659073,1.031105,1.063177,67.508200,4.880234,23.816680,66.946780,4.318809,18.652114,66.038018,3.410050,66.378924,3.750956,64.097279,67.962727,67.032037,66.364014,3.736046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,21422666,51.102397,49.896027,-1.206371,1.455330,49.362335,-1.740062,3.027816,53.252010,2.149613,4.620836,50.836791,-0.265607,48.002100,-3.100297,47.267155,46.288363,50.464850,48.006789,-3.095608
1087,21609582,73.242580,71.538100,-1.704479,2.905248,70.784570,-2.458011,6.041819,70.616974,-2.625606,6.893807,70.979881,-2.262699,72.353162,-0.889418,73.732138,71.876193,71.472038,72.360123,-0.882457
1088,22882299,53.880479,53.860794,-0.019685,0.000388,51.119675,-2.760805,7.622043,52.539883,-1.340597,1.797200,52.506784,-1.373695,50.020962,-3.859518,52.115465,48.387421,49.603358,50.035415,-3.845065
1089,22011925,68.198770,66.386430,-1.812342,3.284582,63.151203,-5.047567,25.477936,65.467740,-2.731028,7.458512,65.001791,-3.196979,65.126225,-3.072545,67.432425,62.758503,65.242775,65.144568,-3.054203


In [28]:
print("{:13} : {:14}  |  {:14}  |  {:13}  |   {:13}  |  {:}  |  {:} ".format('Modality', 'MAE (DECON)', 'MAE (DECON B)', "MSE (DECON)", "MSE (DECON B)",  "PR (DECON)", "PR (DECON B)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:.3f} ".format(
        names[idx],
        
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].mean_age_delta_decon).mean(), np.abs(dfs[idx].mean_age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        
        np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age_decon)[0],
    )
         )

Modality      : MAE (DECON)     |  MAE (DECON B)   |  MSE (DECON)    |   MSE (DECON B)  |  PR (DECON)  |  PR (DECON B) 
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  2.743 ± 0.061  |   2.743 ± 0.061   |   11.584 ± 0.485  |   11.583 ± 0.485   |   0.912   |   0.912 
T1_linear     :  2.822 ± 0.065  |   2.825 ± 0.065   |   12.600 ± 0.558  |   12.621 ± 0.559   |   0.905   |   0.905 
jacobian      :  4.414 ± 0.098  |   4.415 ± 0.098   |   29.994 ± 1.241  |   30.014 ± 1.242   |   0.810   |   0.810 
vbm           :  3.269 ± 0.074  |   3.269 ± 0.074   |   16.633 ± 0.680  |   16.632 ± 0.680   |   0.880   |   0.880 
T2_nonlinear  :  2.639 ± 0.061  |   2.641 ± 0.061   |   11.051 ± 0.476  |   11.074 ± 0.478   |   0.916   |   0.915 
T2_lesions    :  6.088 ± 0.136  |   6.088 ± 0.136   |   57.235 ± 2.355  |   57.252 ± 2.355   |   0.707   |   0.707 
swi           :  3.927 ± 0.092  |   3.927 ± 0.092   |   24.606 ± 1.102  |   24.607 ± 1.1